# ORM querying examples

In this tutorial, we will get to see more examples of ORM querying. 

First, we will gather a lot of data. In order to achieve that we will write a randomized experiment for grasping a couple of objects.
In the experiment the robot will try to grasp a randomized object using random poses and torso heights.


In [1]:
from pycram.designators.action_designator import MoveTorsoActionPerformable, ParkArmsActionPerformable

Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']
Failed to import Giskard messages, the real robot will not be available
Could not import RoboKudo messages, RoboKudo interface could not be initialized
pybullet build time: Nov 28 2023 23:51:11


RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xf . Check the section C-API incompatibility at the Troubleshooting ImportError section at https://numpy.org/devdocs/user/troubleshooting-importerror.html#c-api-incompatibility for indications on how to solve this problem .

ImportError: numpy.core.multiarray failed to import

In [2]:
from pycram.designators.action_designator import MoveTorsoActionPerformable, ParkArmsActionPerformable
from tf import transformations
import itertools
import time
from typing import Optional, List, Tuple

import numpy as np

import sqlalchemy.orm
import tf
import tqdm

import pycram.orm.base
import pycram.tasktree
from pycram.worlds.bullet_world import BulletWorld, Object as BulletWorldObject
from pycram.designators.action_designator import MoveTorsoAction, PickUpAction, NavigateAction, ParkArmsAction
from pycram.designators.object_designator import ObjectDesignatorDescription
import pycram.datastructures.enums
from pycram.plan_failures import PlanFailure
from pycram.process_module import ProcessModule
from pycram.datastructures.enums import WorldMode
from pycram.process_module import simulated_robot
import sqlalchemy.orm
# from pycram.resolver.location.jpt_location import JPTCostmapLocation
import pycram.orm
from pycram.orm.base import Position, RobotState
from pycram.orm.tasktree import TaskTreeNode
from pycram.language import Code
from pycram.orm.action_designator import PickUpAction as ORMPickUpAction
from pycram.orm.object_designator import Object
import sqlalchemy.sql
import pandas as pd

from pycram.datastructures.pose import Pose

np.random.seed(420)

ProcessModule.execution_delay = False
pycram.orm.base.ProcessMetaData().description = "Tutorial for learning from experience in a Grasping action."


class GraspingExplorer:
    """Class to try randomized grasping plans."""

    world: Optional[BulletWorld]

    def __init__(self, robots: Optional[List[Tuple[str, str]]] = None, objects: Optional[List[Tuple[str, str]]] = None,
                 arms: Optional[List[str]] = None, grasps: Optional[List[str]] = None,
                 samples_per_scenario: int = 1000):
        """
        Create a GraspingExplorer.
        :param robots: The robots to use
        :param objects: The objects to try to grasp
        :param arms: The arms of the robot to use
        :param grasps: The grasp orientations to use
        :param samples_per_scenario: The number of tries per scenario.
        """
        # store exploration space
        if not robots:
            self.robots: List[Tuple[str, str]] = [("pr2", "pr2.urdf")]

        if not objects:
            self.objects: List[Tuple[str, pycram.datastructures.enums.ObjectType, str]] = [("cereal", pycram.datastructures.enums.ObjectType.BREAKFAST_CEREAL, "breakfast_cereal.stl"),
                                                                            ("bowl", pycram.datastructures.enums.ObjectType.BOWL, "bowl.stl"),
                                                                            ("milk", pycram.datastructures.enums.ObjectType.MILK, "milk.stl"),
                                                                            ("spoon", pycram.datastructures.enums.ObjectType.SPOON, "spoon.stl")]

        if not arms:
            self.arms: List[str] = ["left", "right"]

        if not grasps:
            self.grasps: List[str] = ["left", "right", "front", "top"]

        # store trials per scenario
        self.samples_per_scenario: int = samples_per_scenario

        # chain hyperparameters
        self.hyper_parameters = [self.robots, self.objects, self.arms, self.grasps]

        self.total_tries = 0
        self.total_failures = 0

    def perform(self, session: sqlalchemy.orm.Session):
        """
        Perform all experiments.
        :param session: The database-session to insert the samples in.
        """

        # create progress bar
        progress_bar = tqdm.tqdm(
            total=np.prod([len(p) for p in self.hyper_parameters]) * self.samples_per_scenario)

        self.world = BulletWorld(mode=WorldMode.DIRECT)

        # for every robot
        for robot, robot_urdf in self.robots:

            # spawn it
            robot = BulletWorldObject(robot, pycram.datastructures.enums.ObjectType.ROBOT, robot_urdf)

            # for every obj
            for obj, obj_type, obj_stl in self.objects:

                # spawn it
                bw_object = BulletWorldObject(obj, obj_type, obj_stl)

                # create object designator
                object_designator = ObjectDesignatorDescription(names=[obj])

                # for every arm and grasp pose
                for arm, grasp in itertools.product(self.arms, self.grasps):
                    # sample positions in 2D
                    positions = np.random.uniform([-2, -2], [2, 2], (self.samples_per_scenario, 2))

                    # for every position
                    for position in positions:

                        # set z axis to 0
                        position = [*position, 0]

                        # calculate orientation for robot to face the object
                        angle = np.arctan2(position[1], position[0]) + np.pi
                        orientation = list(transformations.quaternion_from_euler(0, 0, angle, axes="sxyz"))

                        # try to execute a grasping plan
                        with simulated_robot:

                            ParkArmsActionPerformable(pycram.datastructures.enums.Arms.BOTH).perform()
                            # navigate to sampled position
                            NavigateAction([Pose(position, orientation)]).resolve().perform()

                            # move torso
                            height = np.random.uniform(0., 0.33, 1)[0]
                            MoveTorsoActionPerformable(height).perform()

                            # try to pick it up
                            try:
                                PickUpAction(object_designator, [arm], [grasp]).resolve().perform()

                            # if it fails
                            except PlanFailure:

                                # update failure stats
                                self.total_failures += 1

                            # reset BulletWorld
                            self.world.reset_world()

                            # update progress bar
                            self.total_tries += 1

                            # insert into database
                            pycram.tasktree.task_tree.insert(session, use_progress_bar=False)
                            pycram.tasktree.reset_tree()

                            progress_bar.update()
                            progress_bar.set_postfix(success_rate=(self.total_tries - self.total_failures) /
                                                                  self.total_tries)

                bw_object.remove()
            robot.remove()


pybullet build time: Nov 28 2023 23:51:11


RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xf . Check the section C-API incompatibility at the Troubleshooting ImportError section at https://numpy.org/devdocs/user/troubleshooting-importerror.html#c-api-incompatibility for indications on how to solve this problem .

ImportError: numpy.core.multiarray failed to import

Next we have to establish a connection to a database and execute the experiment a couple of times. Note that the (few) number of samples we generate is only for demonstrations.
For robust and reliable machine learning millions of samples are required.


In [5]:
engine = sqlalchemy.create_engine("sqlite+pysqlite:///:memory:")
session = sqlalchemy.orm.Session(bind=engine)
pycram.orm.base.Base.metadata.create_all(bind=engine)
session.commit()

explorer = GraspingExplorer(samples_per_scenario=30)
explorer.perform(session)

  0%|                                                                                                                                                                                                       | 0/960 [00:00<?, ?it/s]Unknown tag "rgba_color" in /robot[@name='cereal_object']/link[@name='cereal_main']/visual[1]/material[@name='white']
Unknown tag "rgba_color" in /robot[@name='cereal_object']/link[@name='cereal_main']/visual[1]/material[@name='white']
  0%|▌                                                                                                                                                                              | 3/960 [00:01<08:17,  1.92it/s, success_rate=0]

[INFO] [1720087576.270172]: Ontology [http://www.ease-crc.org/ont/SOMA-HOME.owl#]'s name: SOMA-HOME has been loaded
[INFO] [1720087576.270910]: - main namespace: SOMA-HOME
[INFO] [1720087576.271278]: - loaded ontologies:
[INFO] [1720087576.271586]: http://www.ease-crc.org/ont/SOMA-HOME.owl#
[INFO] [1720087576.272028]: http://www.ease-crc.org/ont/DUL.owl#
[INFO] [1720087576.272324]: http://www.ease-crc.org/ont/SOMA.owl#
[INFO] [1720087576.291415]: Waiting for IK service: /pr2_left_arm_kinematics/get_ik


 25%|███████████████████████████████████████████▎                                                                                                                                 | 240/960 [00:12<00:30, 23.69it/s, success_rate=0]Unknown tag "rgba_color" in /robot[@name='bowl_object']/link[@name='bowl_main']/visual[1]/material[@name='white']
Unknown tag "rgba_color" in /robot[@name='bowl_object']/link[@name='bowl_main']/visual[1]/material[@name='white']
 50%|██████████████████████████████████████████████████████████████████████████████████████▌                                                                                      | 480/960 [00:22<00:20, 23.15it/s, success_rate=0]Unknown tag "rgba_color" in /robot[@name='milk_object']/link[@name='milk_main']/visual[1]/material[@name='white']
Unknown tag "rgba_color" in /robot[@name='milk_object']/link[@name='milk_main']/visual[1]/material[@name='white']
 75%|███████████████████████████████████████████████████████████████████████████████████

spoon problem


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 960/960 [00:44<00:00, 22.30it/s, success_rate=0]

In [1]:
ins = sqlalchemy.inspect(engine)
ins.get_table_names()

NameError: name 'sqlalchemy' is not defined

In [13]:
inspector = sqlalchemy.inspect(engine)
inspector.get_table_names()

['AccessingMotion',
 'Action',
 'BelieveObject',
 'CloseAction',
 'ClosingMotion',
 'Color',
 'Designator',
 'DetectAction',
 'DetectingMotion',
 'FaceAtAction',
 'GraspingAction',
 'GripAction',
 'LookAtAction',
 'LookingMotion',
 'Motion',
 'MoveGripperMotion',
 'MoveMotion',
 'MoveTCPMotion',
 'MoveTorsoAction',
 'NavigateAction',
 'Object',
 'ObjectPart',
 'OpenAction',
 'OpeningMotion',
 'ParkArmsAction',
 'PickUpAction',
 'PlaceAction',
 'Pose',
 'Position',
 'ProcessMetaData',
 'Quaternion',
 'Release',
 'RobotState',
 'SetGripperAction',
 'TaskTreeNode',
 'TransportAction',
 'WorldStateDetectingMotion']

In [19]:
from pycram.orm.utils import write_database_to_file
write_database_to_file(sqlalchemy.orm.sessionmaker(bind=engine),filename="pycram.db")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 960/960 [42:46<00:00,  2.67s/it, success_rate=0]


In [15]:
from sqlalchemy import select
import pandas as pd
query = session.query(ORMPickUpAction)
stmt = query.statement
df = pd.read_sql(stmt, engine)

In [16]:
df.head(20)

,process_metadata_id,id,id_1,id_2,action_dtype,dtype,robot_state_id,arm,grasp,object_id
0,1,5,5,5,PickUpAction,PickUpAction,4,left,left,1
1,1,11,11,11,PickUpAction,PickUpAction,8,left,left,2
2,1,17,17,17,PickUpAction,PickUpAction,12,left,left,3
3,1,23,23,23,PickUpAction,PickUpAction,16,left,left,4
4,1,29,29,29,PickUpAction,PickUpAction,20,left,left,5
5,1,35,35,35,PickUpAction,PickUpAction,24,left,left,6
6,1,41,41,41,PickUpAction,PickUpAction,28,left,left,7
7,1,47,47,47,PickUpAction,PickUpAction,32,left,left,8
8,1,53,53,53,PickUpAction,PickUpAction,36,left,left,9
9,1,59,59,59,PickUpAction,PickUpAction,40,left,left,10


The success_rate of the output above indicates how many of our samples succeeded in trying to grasp a randomized object.

Now that we have data to query from and a running session, we can actually start creating queries. 
Let's say we want to select positions of robots that were able to grasp a specific object (in this case a "milk" object):

In [20]:
from sqlalchemy import select
from pycram.datastructures.enums import ObjectType

milk = BulletWorldObject("Milk", ObjectType.MILK, "milk.stl")

# query all relative robot positions in regard to an objects position
# make sure to order the joins() correctly
query = (select(ORMPickUpAction.arm, ORMPickUpAction.grasp, RobotState.torso_height, Position.x, Position.y)
         # .join(TaskTreeNode.code)
         # .join(Code.designator.of_type(ORMPickUpAction))
         .join(ORMPickUpAction.robot_state)
         .join(RobotState.pose)
         .join(pycram.orm.base.Pose.position)
         .join(ORMPickUpAction.object).where(Object.obj_type == milk.obj_type)
                                      .where(TaskTreeNode.status == "SUCCEEDED"))
print(query)

df = pd.read_sql_query(query, session.get_bind())
print(df.head())

[ERROR] [1720085536.003958]: An object with the name Milk already exists in the world.


AttributeError: 'Object' object has no attribute 'obj_type'

If you are not familiar with sqlalchemy querying you might wonder what the of_type() function does and why we needed it in this query:

In order to understand the importance of the of_type() function in the joins above it is crucial to understand the inheritance structure in the ORM package. The action necessary for this query is the PickUpAction. It inherits the Action class/table (which holds all the actions). The Action class itself on the other hand inherits Designator (which holds all the actions, but also all the motions). 
We started our joins by joining TaskTreeNode on its relationship to Code and Code on its relationship to Designator. Next table we need is the PickUpAction table, but there is no specified relationship between Designator and PickUpAction. But we do know that a PickUpAction is actually a Designator, meaning, it inherits from Designator. So we can just "tell" the join to join Code on every Designator, that is "of_type" PickUpAction (.join(Code.designator.of_type(ORMPickUpAction))). 
The effect of this function can also be seen in the printed query of above's output. 

Another interesting query: Let's say we want to select the torso height and positions of robots relative to the object they were trying to grasp:

In [21]:
from pycram.orm.base import Pose as ORMPose

robot_pose = sqlalchemy.orm.aliased(ORMPose)
object_pose = sqlalchemy.orm.aliased(ORMPose)
robot_position = sqlalchemy.orm.aliased(Position)
object_position = sqlalchemy.orm.aliased(Position)

query = (select(TaskTreeNode.status, Object.obj_type, 
                       sqlalchemy.label("relative torso height", object_position.z - RobotState.torso_height),
                       sqlalchemy.label("x", robot_position.x - object_position.x),
                       sqlalchemy.label("y", robot_position.y - object_position.y))
         # .join(TaskTreeNode.code)
         # .join(Code.designator.of_type(ORMPickUpAction))
         .join(ORMPickUpAction.robot_state)
         .join(robot_pose, RobotState.pose)
         .join(robot_position, robot_pose.position)
         .join(ORMPickUpAction.object)
         .join(object_pose, Object.pose)
         .join(object_position, object_pose.position))
print(query)

df = pd.read_sql(query, session.get_bind())
df["status"] = df["status"].apply(lambda x: str(x.name))
print(df.head())

SELECT "TaskTreeNode".status, "Object".obj_type, "Position_1".z - "RobotState".torso_height AS "relative torso height", "Position_2".x - "Position_1".x AS x, "Position_2".y - "Position_1".y AS y 
FROM "Designator" JOIN "Action" ON "Designator".id = "Action".id JOIN "PickUpAction" ON "Action".id = "PickUpAction".id JOIN "RobotState" ON "RobotState".id = "Action".robot_state_id JOIN "Pose" AS "Pose_1" ON "Pose_1".id = "RobotState".pose_id JOIN "Position" AS "Position_2" ON "Position_2".id = "Pose_1".position_id JOIN "Object" ON "Object".id = "PickUpAction".object_id JOIN "Pose" AS "Pose_2" ON "Pose_2".id = "Object".pose_id JOIN "Position" AS "Position_1" ON "Position_1".id = "Pose_2".position_id, "TaskTreeNode"


/home/malineni/.local/lib/python3.8/site-packages/pandas/io/sql.py:1563: SAWarning: SELECT statement has a cartesian product between FROM element(s) "TaskTreeNode" and FROM element "Action".  Apply join condition(s) between each element to resolve.
  return self.con.execute(sql, *args)


      status                     obj_type  relative torso height         x  \
0    RUNNING  ObjectType.BREAKFAST_CEREAL              -0.269995 -0.737416   
1  SUCCEEDED  ObjectType.BREAKFAST_CEREAL              -0.269995 -0.737416   
2  SUCCEEDED  ObjectType.BREAKFAST_CEREAL              -0.269995 -0.737416   
3  SUCCEEDED  ObjectType.BREAKFAST_CEREAL              -0.269995 -0.737416   
4     FAILED  ObjectType.BREAKFAST_CEREAL              -0.269995 -0.737416   

          y  
0 -0.187877  
1 -0.187877  
2 -0.187877  
3 -0.187877  
4 -0.187877  


Obviously the query returned every row of the database since we didn't apply any filters.

Why is this query interesting? This query not only required more joins and the usage of the of_type() function, but we actually needed to access two of the tables twice with different purposes, namely the Pose and Position tables. We wanted to get the position of the robot relative to the object position, meaning we had to obtain all robot positions and all object positions. If we want to access the same table twice, we have to make sure to rename (one of) the occurrences in our query in order to provide proper sql syntax. This can be done by creating aliases using the sqlalchemy.orm.aliased() function. Sqlalchemy will automatically rename all the aliased tables for you during runtime.